In [1]:
pip install Faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.4 MB/s 


In [ ]:
import numpy as np
import pandas as pd
from faker import Faker

In [ ]:
# Rread files
employees = pd.read_excel('EmployeeRoles.xlsx')
permissions = pd.read_excel('RolePermissions.xlsx')

In [ ]:
# remove the admins and keep just the 'Name', 'Role', 'Permission', and 'Level' columns of the df
admins = permissions[permissions['Role'] == 'Administrator']['Name'].unique().tolist()
standard_permissions = permissions[~permissions['Name'].isin(admins)][['Name', 'Role', 'Permission', 'Level']]
standard_permissions.head(10)

In [ ]:
# obfuscate the data
faker = Faker()
e = employees['Name'].unique()
d = {}
for employee in e:
  d[employee] = faker.name()

employees = employees.replace({"Name": d})

employees['Role'] = employees['Role'].str.replace('Mesa','Company')
employees['Role'] = employees['Role'].str.replace('Agena','Entity')
permissions = permissions.rename(columns={'Name':'Role'})
permissions['Role'] = permissions['Role'].str.replace('Mesa','Company')
permissions['Role'] = permissions['Role'].str.replace('Agena','Entity')

In [ ]:
# combine the employee file and the permissions file
all_permissions = employees.merge(permissions, how='outer', on='Role')

In [ ]:
# some users have multiple roles, each with a differnt permissions level. we need to replace the pmermission levels with a binary (1=edit access or 0=no edit access); those values will be summed when we pivot the data; a user that has a permission that sums > 0 has access to that permission across any of their roles
df = all_permissions.replace({'Level': {'Full': 1, 'Edit': 1, 'Create': 1, 'View': 0, 'None': 0}})

# pivot the all_permissions df to get the permissions in the column headers; the index variable is all of the columns that we need to keep from the raw data
df = df.pivot_table(index='Name', values='Level', columns='Permission', aggfunc='sum').reset_index().rename_axis(None, axis=1).fillna(0)
for col in df.columns:
  if col != 'Name':
    df[col] = np.where(df[col] > 0, 'Yes', 'No')
df.head()

,Name,4599 GL Export Custom Field,4599 Transaction Child,AP Eligible Customer,AP Eligible Order,AP Minimum Purchase,AP Promotion Discount,AP Promotion Shipping Method,AP Promotion Shipping Price,AP Promotion Tier,...,Web Site Report,Web Store Report,Website (External) publisher,Work Calendar,Work Order,Work Order Close,Work Order Completion,Work Order Issue,Workflow,Zengin Bank Fee Schedule
0,Aaron Peterson,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
1,Aaron Wheeler,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
2,Adam Edwards,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
3,Alan Watson,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
4,Albert Wong,No,No,No,No,No,No,No,No,No,...,No,No,No,No,Yes,Yes,Yes,No,No,No


In [ ]:
# pivot the all_permissions df with the role included for presentation purposes; end user will want to see which roles each user has and the level of their permissions
df2 = all_permissions.pivot_table(index=['Name','Role'], values='Level', columns='Permission', aggfunc=','.join).reset_index().rename_axis(None, axis=1).fillna(0).replace({0 : 'None'})
df2.head()

,Name,Role,4599 GL Export Custom Field,4599 Transaction Child,AP Eligible Customer,AP Eligible Order,AP Minimum Purchase,AP Promotion Discount,AP Promotion Shipping Method,AP Promotion Shipping Price,...,Web Site Report,Web Store Report,Website (External) publisher,Work Calendar,Work Order,Work Order Close,Work Order Completion,Work Order Issue,Workflow,Zengin Bank Fee Schedule
0,Aaron Peterson,Company Sales,None,None,None,None,None,None,None,None,...,None,None,None,View,View,None,View,View,None,None
1,Aaron Wheeler,Company Shipping & Receiving Clerk,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Adam Edwards,Company Accounting - Cost Accountant,None,None,None,None,None,None,None,None,...,None,None,None,None,View,View,View,View,None,None
3,Adam Edwards,Company Fixed Assets Management,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Adam Edwards,Company Production View,None,None,None,None,None,None,None,None,...,None,None,None,None,View,View,View,View,None,None


In [ ]:
# filter the df to those users with the offending permissions
tbld = 'Users that can create journal entries (Make Journal Entry) and approve journal entries (Journal Approval).'
vf = pd.unique(df[(df['Make Journal Entry'] == "Yes") & (df['Journal Approval'] == "Yes")]['Name']).tolist()
lf = df2[df2['Name'].isin(vf)][['Name', 'Role', 'Make Journal Entry', 'Journal Approval']]
lf.to_excel('Create Approve JE.xlsx')

In [ ]:
# filter the df to those users with the offending permissions
tbld = 'Users that can create customer invoices (Invoice) and can either receive customer payments (Customer Deposit) or record customer payments (Customer Payment).'
vf = pd.unique(df[(df['Invoice'] == "Yes") & ((df['Customer Deposit'] == "Yes") | (df['Customer Payment'] == "Yes"))]['Name']).tolist()
lf = df2[df2['Name'].isin(vf)][['Name', 'Role', 'Invoice', 'Customer Deposit', 'Customer Payment']]
lf.to_excel('Invoice Customer Payment.xlsx')

In [ ]:
# filter the df to those users with the offending permissions
tbld = 'Users that can create vendors (Vendors) and pay vendors (Pay Bills).'
vf = pd.unique(df[(df['Vendors'] == "Yes") & (df['Pay Bills'] == "Yes")]['Name']).tolist()
lf = df2[df2['Name'].isin(vf)][['Name', 'Role', 'Customers', 'Customer Refund']]
lf.to_excel('Vendors Pay Bills.xlsx')

In [ ]:
# filter the df to those users with the offending permissions
tbld = 'Users that can create credit memos (Credit Memo) and can either receive customer payments (Customer Deposit) or record customer payments (Customer Payment).'
vf = pd.unique(df[(df['Credit Memo'] == "Yes") & ((df['Customer Deposit'] == "Yes") | (df['Customer Payment'] == "Yes"))]['Name']).tolist()
lf = df2[df2['Name'].isin(vf)][['Name', 'Role', 'Credit Memo', 'Customer Deposit', 'Customer Payment']]
lf.to_excel(f'Credit Memo Customer Payment.xlsx')

In [ ]:
# filter the df to those users with the offending permissions
tbld = 'Users that can create customers (Customers) and issue customer refunds (Customer Refund).'
vf = pd.unique(df[(df['Customers'] == "Yes") & (df['Customer Refund'] == "Yes")]['Name']).tolist()
lf = df2[df2['Name'].isin(vf)][['Name', 'Role', 'Customers', 'Customer Refund']]
lf.to_excel(f'Customers Customer Refund.xlsx')

In [ ]:
# filter the df to those users with the offending permissions
tbld = 'Users that can create customers (Customers) and credit memos (Credit Memo).'
vf = pd.unique(df[(df['Customers'] == "Yes") & (df['Credit Memo'] == "Yes")]['Name']).tolist()
lf = df2[df2['Name'].isin(vf)][['Name', 'Role', 'Customers', 'Credit Memo']]
lf.to_excel('Customers Credit Memo.xlsx')